In [39]:
import pandas as pd

In [40]:
df = pd.read_csv("input.csv")

In [41]:
from datetime import datetime
import numpy as np
import pandas as pd

def parse_date(date_str):
    """
    解析日期字串，返回標準格式的日期字串和數值
    支援各種日期格式
    """
    if pd.isna(date_str) or date_str == '' or str(date_str).strip() == '':
        return None, None
    
    date_str = str(date_str).strip()
    
    # 嘗試各種日期格式（包含時間戳記）
    date_formats = [
        '%Y-%m-%d %H:%M:%S',  # 2025-09-20 00:00:00
        '%Y/%m/%d %H:%M:%S',  # 2025/09/20 00:00:00
        '%Y-%m-%d',           # 2025-09-20
        '%Y/%m/%d',           # 2025/09/20
        '%Y-%m',              # 2025-09
        '%Y/%m',              # 2025/09
        '%Y',                 # 2025
    ]
    
    for fmt in date_formats:
        try:
            dt = datetime.strptime(date_str, fmt)
            # 返回標準格式的日期字串和用於比較的數值
            date_num = int(dt.strftime('%Y%m%d'))
            date_str_formatted = dt.strftime('%Y-%m-%d')
            return date_str_formatted, date_num
        except ValueError:
            continue
    
    return None, None

def find_min_max_dates(row):
    """
    找出最早和最晚的出版日期
    - 如果只有一個書商有日期 → min = max = 該日期
    - 如果有多個書商有日期 → 比較找出 min 和 max
    """
    dates_info = []
    
    # 收集所有有效的日期
    for col in ['bookscom_publish_date', 'kobo_publish_date', 'readmoo_publish_date']:
        date_str, date_num = parse_date(row[col])
        if date_str is not None:
            dates_info.append((date_str, date_num))
    
    # 如果沒有任何有效日期
    if not dates_info:
        return None, None
    
    # 如果只有一個日期源
    if len(dates_info) == 1:
        date_str = dates_info[0][0]
        return date_str, date_str  # min = max = 該日期
    
    # 如果有多個日期源，進行比較
    dates_info.sort(key=lambda x: x[1])  # 按數值排序
    min_date = dates_info[0][0]   # 最早的日期
    max_date = dates_info[-1][0]  # 最晚的日期
    
    return min_date, max_date

# 應用函數找出 min 和 max 日期
print("🔄 開始處理日期...")
df[['min_publish_date', 'max_publish_date']] = df.apply(
    lambda row: pd.Series(find_min_max_dates(row)), axis=1
)

# 統計結果
total_rows = len(df)
rows_with_dates = df['min_publish_date'].notna().sum()
rows_with_single_date = (df['min_publish_date'] == df['max_publish_date']).sum()
rows_with_multiple_dates = rows_with_dates - rows_with_single_date

print(f"\n✅ 日期處理完成！")
print(f"  - 總筆數: {total_rows}")
print(f"  - 有日期的筆數: {rows_with_dates} ({rows_with_dates/total_rows*100:.1f}%)")
print(f"  - 單一日期源: {rows_with_single_date} 筆")
print(f"  - 多個日期源: {rows_with_multiple_dates} 筆")

print("\n📊 前 10 筆範例:")
print(df[['bookscom_publish_date', 'kobo_publish_date', 'readmoo_publish_date', 
          'min_publish_date', 'max_publish_date']].head(10))


🔄 開始處理日期...

✅ 日期處理完成！
  - 總筆數: 11269
  - 有日期的筆數: 11267 (100.0%)
  - 單一日期源: 11053 筆
  - 多個日期源: 214 筆

📊 前 10 筆範例:
  bookscom_publish_date    kobo_publish_date readmoo_publish_date  \
0                   NaN                  NaN  2025-09-20 00:00:00   
1                   NaN                  NaN  2025-09-09 00:00:00   
2                   NaN  2025-07-14 00:00:00  2025-07-15 00:00:00   
3                   NaN  2025-07-14 00:00:00                  NaN   
4                   NaN                  NaN  2025-09-19 00:00:00   
5   2025-09-10 00:00:00                  NaN                  NaN   
6                   NaN  2025-09-25 00:00:00                  NaN   
7                   NaN  2025-07-15 00:00:00  2025-01-17 00:00:00   
8                   NaN  2025-07-17 00:00:00                  NaN   
9                   NaN  2025-09-03 00:00:00                  NaN   

  min_publish_date max_publish_date  
0       2025-09-20       2025-09-20  
1       2025-09-09       2025-09-09  
2       2025

In [42]:
merge_df = pd.read_csv('/Users/alioth1225/Documents/College/merge/clustering/final_merged_bert_processed.csv')

In [43]:
from datetime import datetime
import re

def split_taicca_ids(taicca_id_str):
    """
    拆分 TAICCA_ID，支援「/」分隔
    例如: "E253-00004 / E253-00003" -> ["E253-00004", "E253-00003"]
    """
    if pd.isna(taicca_id_str) or str(taicca_id_str).strip() == '':
        return []
    
    # 用「/」分隔，並去除前後空白
    ids = [id.strip() for id in str(taicca_id_str).split('/')]
    # 過濾掉空字串
    ids = [id for id in ids if id]
    return ids

def parse_date_to_datetime(date_str):
    """
    將日期字串轉換成 datetime 物件，方便比較
    """
    if pd.isna(date_str) or str(date_str).strip() == '':
        return None
    
    date_str = str(date_str).strip()
    
    # 嘗試各種日期格式
    date_formats = [
        '%Y-%m-%d %H:%M:%S',
        '%Y/%m/%d %H:%M:%S',
        '%Y-%m-%d',
        '%Y/%m/%d',
        '%Y-%m',
        '%Y/%m',
        '%Y',
    ]
    
    for fmt in date_formats:
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    
    return None

def find_dates_for_merge_row(row, source_df):
    """
    根據 merge_df 的某一列，查詢 source_df (df) 中的日期資料
    
    處理流程：
    1. 拆分 NEW_TAICCA_ID（如果有多個用「/」分隔）
    2. 對每個 ID 去 source_df 查詢
    3. 收集所有查到的 min_publish_date 和 max_publish_date
    4. 返回所有日期中最早和最晚的
    """
    # 拆分 TAICCA_ID
    taicca_ids = split_taicca_ids(row['NEW_TAICCA_ID'])
    
    if not taicca_ids:
        return None, None
    
    all_min_dates = []
    all_max_dates = []
    
    # 對每個 ID 查詢
    for taicca_id in taicca_ids:
        # 在 source_df 中查詢該 ID
        matched_rows = source_df[source_df['NEW_TAICCA_ID'] == taicca_id]
        
        # 收集該 ID 的所有日期
        for _, matched_row in matched_rows.iterrows():
            min_date = parse_date_to_datetime(matched_row.get('min_publish_date'))
            max_date = parse_date_to_datetime(matched_row.get('max_publish_date'))
            
            if min_date is not None:
                all_min_dates.append(min_date)
            if max_date is not None:
                all_max_dates.append(max_date)
    
    # 如果沒有找到任何有效日期
    if not all_min_dates and not all_max_dates:
        return None, None
    
    # 找出最早和最晚的日期
    final_min = min(all_min_dates).strftime('%Y-%m-%d') if all_min_dates else None
    final_max = max(all_max_dates).strftime('%Y-%m-%d') if all_max_dates else None
    
    return final_min, final_max

# 測試函數
print("🧪 測試拆分 TAICCA_ID:")
test_id = "E253-00004 / E253-00003"
print(f"  輸入: {test_id}")
print(f"  輸出: {split_taicca_ids(test_id)}")

print("\n🧪 測試查詢第一筆資料:")
if len(merge_df) > 0:
    first_row = merge_df.iloc[0]
    print(f"  NEW_TAICCA_ID: {first_row['NEW_TAICCA_ID']}")
    min_date, max_date = find_dates_for_merge_row(first_row, df)
    print(f"  查詢結果:")
    print(f"    min_publish_date: {min_date}")
    print(f"    max_publish_date: {max_date}")


🧪 測試拆分 TAICCA_ID:
  輸入: E253-00004 / E253-00003
  輸出: ['E253-00004', 'E253-00003']

🧪 測試查詢第一筆資料:
  NEW_TAICCA_ID: E253-00004 / E253-00003
  查詢結果:
    min_publish_date: 2025-07-14
    max_publish_date: 2025-07-15


In [44]:
from tqdm import tqdm

print("🔄 開始為 merge_df 填入日期資料...")
print(f"總共需要處理 {len(merge_df)} 筆資料\n")

# 使用 tqdm 顯示進度條
min_dates = []
max_dates = []

for idx, row in tqdm(merge_df.iterrows(), total=len(merge_df), desc="處理中"):
    min_date, max_date = find_dates_for_merge_row(row, df)
    min_dates.append(min_date)
    max_dates.append(max_date)

# 將結果填入 merge_df
merge_df['min_publish_date'] = min_dates
merge_df['max_publish_date'] = max_dates

# 統計結果
total = len(merge_df)
filled_min = merge_df['min_publish_date'].notna().sum()
filled_max = merge_df['max_publish_date'].notna().sum()
not_filled = total - filled_min

print(f"\n✅ 日期填入完成！")
print(f"  - 總筆數: {total}")
print(f"  - 成功填入 min_publish_date: {filled_min} ({filled_min/total*100:.1f}%)")
print(f"  - 成功填入 max_publish_date: {filled_max} ({filled_max/total*100:.1f}%)")
print(f"  - 無法填入: {not_filled} ({not_filled/total*100:.1f}%)")

# 顯示一些範例
print("\n📋 查看前 10 筆結果:")
print(merge_df[['NEW_TAICCA_ID', 'min_publish_date', 'max_publish_date']].head(10))


🔄 開始為 merge_df 填入日期資料...
總共需要處理 9276 筆資料



處理中: 100%|██████████| 9276/9276 [00:04<00:00, 2312.95it/s]


✅ 日期填入完成！
  - 總筆數: 9276
  - 成功填入 min_publish_date: 9275 (100.0%)
  - 成功填入 max_publish_date: 9275 (100.0%)
  - 無法填入: 1 (0.0%)

📋 查看前 10 筆結果:
             NEW_TAICCA_ID min_publish_date max_publish_date
0  E253-00004 / E253-00003       2025-07-14       2025-07-15
1               E253-00010       2025-09-03       2025-09-03
2               E253-00011       2025-09-03       2025-09-03
3               E253-00050       2025-08-08       2025-08-18
4               E253-05218       2025-08-08       2025-08-18
5               E253-05219       2025-08-08       2025-08-18
6               E253-00493       2025-09-12       2025-09-12
7               E253-00494       2025-09-12       2025-09-12
8               E253-00495       2025-09-12       2025-09-12
9               E253-00496       2025-09-12       2025-09-12


In [45]:
# 顯示詳細範例
print("=" * 80)
print("📊 詳細範例分析")
print("=" * 80)

# 找出有多個 ID 的資料（包含「/」）
multi_id_mask = merge_df['NEW_TAICCA_ID'].astype(str).str.contains('/', na=False)
multi_id_rows = merge_df[multi_id_mask]

if len(multi_id_rows) > 0:
    print(f"\n✅ 找到 {len(multi_id_rows)} 筆包含多個 ID 的資料")
    print("\n📋 範例 1: 多個 ID 的資料")
    print("-" * 80)
    
    # 顯示第一筆多 ID 的詳細資訊
    example = multi_id_rows.iloc[0]
    print(f"NEW_TAICCA_ID: {example['NEW_TAICCA_ID']}")
    
    # 拆分並查詢每個 ID
    ids = split_taicca_ids(example['NEW_TAICCA_ID'])
    print(f"拆分後的 ID: {ids}")
    
    print("\n各 ID 在 df 中的日期資料:")
    for taicca_id in ids:
        matched = df[df['NEW_TAICCA_ID'] == taicca_id]
        if len(matched) > 0:
            for _, row in matched.iterrows():
                print(f"  - {taicca_id}:")
                print(f"      min_publish_date: {row.get('min_publish_date')}")
                print(f"      max_publish_date: {row.get('max_publish_date')}")
    
    print(f"\n合併後的日期:")
    print(f"  - min_publish_date: {example['min_publish_date']}")
    print(f"  - max_publish_date: {example['max_publish_date']}")
else:
    print("\n⚠️  沒有找到包含多個 ID 的資料")

# 找出單一 ID 的範例
single_id_mask = ~multi_id_mask & merge_df['NEW_TAICCA_ID'].notna()
single_id_rows = merge_df[single_id_mask]

if len(single_id_rows) > 0:
    print(f"\n\n📋 範例 2: 單一 ID 的資料")
    print("-" * 80)
    
    example = single_id_rows.iloc[0]
    print(f"NEW_TAICCA_ID: {example['NEW_TAICCA_ID']}")
    
    # 查詢這個 ID 在 df 中的資料
    matched = df[df['NEW_TAICCA_ID'] == example['NEW_TAICCA_ID']]
    if len(matched) > 0:
        print(f"\n在 df 中找到 {len(matched)} 筆資料:")
        for idx, row in matched.iterrows():
            print(f"  - min_publish_date: {row.get('min_publish_date')}")
            print(f"  - max_publish_date: {row.get('max_publish_date')}")
    
    print(f"\nmerge_df 中的日期:")
    print(f"  - min_publish_date: {example['min_publish_date']}")
    print(f"  - max_publish_date: {example['max_publish_date']}")

print("\n" + "=" * 80)

# 儲存結果
output_file = "final_merged_with_dates.csv"
merge_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n✅ 已儲存處理後的 merge_df 至: {output_file}")
print(f"   總筆數: {len(merge_df)}")
print(f"   欄位數: {len(merge_df.columns)}")

# 顯示統計摘要
print("\n📊 最終統計:")
print(f"  - 有 min_publish_date 的筆數: {merge_df['min_publish_date'].notna().sum()}")
print(f"  - 有 max_publish_date 的筆數: {merge_df['max_publish_date'].notna().sum()}")
print(f"  - 缺少日期的筆數: {merge_df['min_publish_date'].isna().sum()}")

# 檢查是否有 min > max 的異常情況
if (merge_df['min_publish_date'].notna() & merge_df['max_publish_date'].notna()).any():
    merge_df['min_date_dt'] = pd.to_datetime(merge_df['min_publish_date'], errors='coerce')
    merge_df['max_date_dt'] = pd.to_datetime(merge_df['max_publish_date'], errors='coerce')
    
    anomaly = merge_df[merge_df['min_date_dt'] > merge_df['max_date_dt']]
    if len(anomaly) > 0:
        print(f"\n⚠️  發現 {len(anomaly)} 筆異常資料（min_date > max_date）")
    else:
        print(f"\n✅ 沒有發現異常資料（所有 min_date <= max_date）")
    
    # 清理臨時欄位
    merge_df = merge_df.drop(['min_date_dt', 'max_date_dt'], axis=1)


📊 詳細範例分析

✅ 找到 1993 筆包含多個 ID 的資料

📋 範例 1: 多個 ID 的資料
--------------------------------------------------------------------------------
NEW_TAICCA_ID: E253-00004 / E253-00003
拆分後的 ID: ['E253-00004', 'E253-00003']

各 ID 在 df 中的日期資料:
  - E253-00004:
      min_publish_date: 2025-07-14
      max_publish_date: 2025-07-14
  - E253-00003:
      min_publish_date: 2025-07-14
      max_publish_date: 2025-07-15

合併後的日期:
  - min_publish_date: 2025-07-14
  - max_publish_date: 2025-07-15


📋 範例 2: 單一 ID 的資料
--------------------------------------------------------------------------------
NEW_TAICCA_ID: E253-00010

在 df 中找到 1 筆資料:
  - min_publish_date: 2025-09-03
  - max_publish_date: 2025-09-03

merge_df 中的日期:
  - min_publish_date: 2025-09-03
  - max_publish_date: 2025-09-03


✅ 已儲存處理後的 merge_df 至: final_merged_with_dates.csv
   總筆數: 9276
   欄位數: 59

📊 最終統計:
  - 有 min_publish_date 的筆數: 9275
  - 有 max_publish_date 的筆數: 9275
  - 缺少日期的筆數: 1

✅ 沒有發現異常資料（所有 min_date <= max_date）


In [46]:
merge_df[['min_publish_date', 'max_publish_date',
       'bookscom_publish_date', 'kobo_publish_date', 'readmoo_publish_date']].head()

,min_publish_date,max_publish_date,bookscom_publish_date,kobo_publish_date,readmoo_publish_date
0,2025-07-14,2025-07-15,NaN,2025-07-14 00:00:00,2025-07-15 00:00:00
1,2025-09-03,2025-09-03,NaN,2025-09-03 00:00:00,NaN
2,2025-09-03,2025-09-03,NaN,NaN,2025-09-03 00:00:00
3,2025-08-08,2025-08-18,NaN,2025-08-18 00:00:00,2025-08-08 00:00:00
4,2025-08-08,2025-08-18,NaN,2025-08-18 00:00:00,2025-08-08 00:00:00


In [47]:
def find_max_price_for_row(row, source_df):
    """
    根據 merge_df 的某一列，查詢 source_df (df) 中的價格資料
    
    處理流程：
    1. 拆分 NEW_TAICCA_ID（如果有多個用「/」分隔）
    2. 對每個 ID 去 source_df 查詢
    3. 收集所有查到的 bookscom_original_price, kobo_original_price, readmoo_original_price
    4. 返回所有價格中的最大值
    """
    # 拆分 TAICCA_ID
    taicca_ids = split_taicca_ids(row['NEW_TAICCA_ID'])
    
    if not taicca_ids:
        return None
    
    all_prices = []
    
    # 對每個 ID 查詢
    for taicca_id in taicca_ids:
        # 在 source_df 中查詢該 ID
        matched_rows = source_df[source_df['NEW_TAICCA_ID'] == taicca_id]
        
        # 收集該 ID 的所有價格
        for _, matched_row in matched_rows.iterrows():
            for price_col in ['bookscom_original_price', 'kobo_original_price', 'readmoo_original_price']:
                price_val = matched_row.get(price_col)
                
                # 嘗試轉換成數值
                if pd.notna(price_val):
                    try:
                        price = float(price_val)
                        if price > 0:  # 只收集正數價格
                            all_prices.append(price)
                    except (ValueError, TypeError):
                        pass
    
    # 如果沒有找到任何有效價格
    if not all_prices:
        return None
    
    # 返回最大值
    return max(all_prices)

# 測試函數
print("🧪 測試價格查詢:")
if len(merge_df) > 0:
    # 找一筆單一 ID 的資料測試
    single_id_mask = ~merge_df['NEW_TAICCA_ID'].astype(str).str.contains('/', na=False)
    single_id_rows = merge_df[single_id_mask & merge_df['NEW_TAICCA_ID'].notna()]
    
    if len(single_id_rows) > 0:
        test_row = single_id_rows.iloc[0]
        print(f"  測試 NEW_TAICCA_ID: {test_row['NEW_TAICCA_ID']}")
        
        # 查詢 df 中的價格
        matched = df[df['NEW_TAICCA_ID'] == test_row['NEW_TAICCA_ID']]
        if len(matched) > 0:
            print(f"\n  在 df 中找到的價格:")
            for _, row in matched.iterrows():
                print(f"    - bookscom_original_price: {row.get('bookscom_original_price')}")
                print(f"    - kobo_original_price: {row.get('kobo_original_price')}")
                print(f"    - readmoo_original_price: {row.get('readmoo_original_price')}")
        
        max_price = find_max_price_for_row(test_row, df)
        print(f"\n  計算出的最大價格: {max_price}")


🧪 測試價格查詢:
  測試 NEW_TAICCA_ID: E253-00010

  在 df 中找到的價格:
    - bookscom_original_price: nan
    - kobo_original_price: 520
    - readmoo_original_price: nan

  計算出的最大價格: 520.0


In [48]:
print("🔄 開始為單一 ID 的資料填入價格...")

# 找出單一 ID 的資料（不包含「/」）
single_id_mask = ~merge_df['NEW_TAICCA_ID'].astype(str).str.contains('/', na=False)
single_id_rows = merge_df[single_id_mask & merge_df['NEW_TAICCA_ID'].notna()]

print(f"找到 {len(single_id_rows)} 筆單一 ID 的資料\n")

# 準備價格列表
prices_to_fill = []

# 使用 tqdm 顯示進度
for idx, row in tqdm(merge_df.iterrows(), total=len(merge_df), desc="處理價格"):
    # 檢查是否為單一 ID
    taicca_id = str(row['NEW_TAICCA_ID'])
    
    if pd.notna(row['NEW_TAICCA_ID']) and '/' not in taicca_id:
        # 單一 ID - 查詢並填入價格
        max_price = find_max_price_for_row(row, df)
        prices_to_fill.append(max_price)
    else:
        # 多個 ID 或空值 - 保留原本的 price
        prices_to_fill.append(row.get('price'))

# 更新 merge_df 的 price 欄位
merge_df['price'] = prices_to_fill

# 統計結果
total = len(merge_df)
single_id_count = len(single_id_rows)
filled_count = merge_df[single_id_mask]['price'].notna().sum()
not_filled_count = single_id_count - filled_count

print(f"\n✅ 價格填入完成！")
print(f"  - 總筆數: {total}")
print(f"  - 單一 ID 筆數: {single_id_count}")
print(f"  - 成功填入價格: {filled_count} ({filled_count/single_id_count*100:.1f}%)")
print(f"  - 無法填入: {not_filled_count} ({not_filled_count/single_id_count*100:.1f}%)")

# 顯示範例
print("\n📋 單一 ID 資料的價格範例（前 10 筆）:")
print(merge_df[single_id_mask][['NEW_TAICCA_ID', 'price']].head(10))


🔄 開始為單一 ID 的資料填入價格...
找到 7283 筆單一 ID 的資料



處理價格: 100%|██████████| 9276/9276 [00:02<00:00, 3719.51it/s]


✅ 價格填入完成！
  - 總筆數: 9276
  - 單一 ID 筆數: 7283
  - 成功填入價格: 7201 (98.9%)
  - 無法填入: 82 (1.1%)

📋 單一 ID 資料的價格範例（前 10 筆）:
   NEW_TAICCA_ID  price
1     E253-00010  520.0
2     E253-00011  520.0
3     E253-00050  630.0
4     E253-05218  315.0
5     E253-05219  315.0
6     E253-00493  700.0
7     E253-00494  700.0
8     E253-00495  700.0
9     E253-00496  700.0
10    E253-00497  700.0


In [49]:
print("=" * 80)
print("📊 價格填補詳細範例")
print("=" * 80)

# 找出單一 ID 的資料
single_id_mask = ~merge_df['NEW_TAICCA_ID'].astype(str).str.contains('/', na=False)
single_id_rows = merge_df[single_id_mask & merge_df['NEW_TAICCA_ID'].notna()]

# 找幾筆有價格的範例
filled_examples = single_id_rows[single_id_rows['price'].notna()].head(3)

print("\n📋 範例：單一 ID 的價格填補")
print("-" * 80)

for idx, example in filled_examples.iterrows():
    print(f"\nNEW_TAICCA_ID: {example['NEW_TAICCA_ID']}")
    
    # 查詢 df 中的價格資料
    matched = df[df['NEW_TAICCA_ID'] == example['NEW_TAICCA_ID']]
    
    if len(matched) > 0:
        print("  在 df 中找到的價格:")
        for _, row in matched.iterrows():
            bookscom_price = row.get('bookscom_original_price')
            kobo_price = row.get('kobo_original_price')
            readmoo_price = row.get('readmoo_original_price')
            
            print(f"    - bookscom_original_price: {bookscom_price if pd.notna(bookscom_price) else 'N/A'}")
            print(f"    - kobo_original_price: {kobo_price if pd.notna(kobo_price) else 'N/A'}")
            print(f"    - readmoo_original_price: {readmoo_price if pd.notna(readmoo_price) else 'N/A'}")
            
            # 收集所有有效價格
            prices = []
            for p in [bookscom_price, kobo_price, readmoo_price]:
                if pd.notna(p):
                    try:
                        prices.append(float(p))
                    except:
                        pass
            
            if prices:
                print(f"    → 所有價格: {prices}")
                print(f"    → 最大值: {max(prices)}")
    
    print(f"\n  填入 merge_df 的價格: {example['price']}")
    print("-" * 80)

print("\n" + "=" * 80)

# 比較多個 ID 的資料（這些應該保留原本的 price）
multi_id_mask = merge_df['NEW_TAICCA_ID'].astype(str).str.contains('/', na=False)
multi_id_rows = merge_df[multi_id_mask]

print(f"\n📊 多個 ID 的資料統計:")
print(f"  - 總筆數: {len(multi_id_rows)}")
print(f"  - 這些資料保留原本的 price 欄位（不受此步驟影響）")

# 顯示價格分布
print(f"\n📈 價格統計:")
print(f"  - 單一 ID 有價格: {single_id_rows['price'].notna().sum()} 筆")
print(f"  - 單一 ID 無價格: {single_id_rows['price'].isna().sum()} 筆")
print(f"  - 多個 ID 有價格: {multi_id_rows['price'].notna().sum()} 筆")
print(f"  - 多個 ID 無價格: {multi_id_rows['price'].isna().sum()} 筆")

if single_id_rows['price'].notna().any():
    print(f"\n💰 單一 ID 價格範圍:")
    print(f"  - 最小值: {single_id_rows['price'].min():.2f}")
    print(f"  - 最大值: {single_id_rows['price'].max():.2f}")
    print(f"  - 平均值: {single_id_rows['price'].mean():.2f}")
    print(f"  - 中位數: {single_id_rows['price'].median():.2f}")

# 儲存最終結果
output_file = "final_merged_with_dates_and_prices.csv"
merge_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"\n" + "=" * 80)
print(f"✅ 已儲存最終結果至: {output_file}")
print(f"   總筆數: {len(merge_df)}")
print(f"   欄位數: {len(merge_df.columns)}")
print("=" * 80)


📊 價格填補詳細範例

📋 範例：單一 ID 的價格填補
--------------------------------------------------------------------------------

NEW_TAICCA_ID: E253-00010
  在 df 中找到的價格:
    - bookscom_original_price: N/A
    - kobo_original_price: 520
    - readmoo_original_price: N/A
    → 所有價格: [520.0]
    → 最大值: 520.0

  填入 merge_df 的價格: 520.0
--------------------------------------------------------------------------------

NEW_TAICCA_ID: E253-00011
  在 df 中找到的價格:
    - bookscom_original_price: N/A
    - kobo_original_price: N/A
    - readmoo_original_price: 520.0
    → 所有價格: [520.0]
    → 最大值: 520.0

  填入 merge_df 的價格: 520.0
--------------------------------------------------------------------------------

NEW_TAICCA_ID: E253-00050
  在 df 中找到的價格:
    - bookscom_original_price: N/A
    - kobo_original_price: 630
    - readmoo_original_price: 630.0
    → 所有價格: [630.0, 630.0]
    → 最大值: 630.0

  填入 merge_df 的價格: 630.0
--------------------------------------------------------------------------------


📊 多個 ID 的資料統計:
  - 

In [50]:
def find_find_values_for_row(row, source_df):
    """
    根據 merge_df 的某一列，查詢 source_df (df) 中的 FIND 欄位資料
    
    處理流程：
    1. 拆分 NEW_TAICCA_ID（如果有多個用「/」分隔）
    2. 對每個 ID 去 source_df 查詢
    3. 檢查該 ID 的「未納入書目FIND」欄位
    4. 如果有值 → 記錄為 "FIND"
    5. 如果沒值 → 記錄為 "空白"
    6. 用「/」連接所有結果
    
    回傳範例：
    - 單一 ID 有 FIND → "FIND"
    - 單一 ID 沒有 FIND → None（留空）
    - 兩個 ID 都有 FIND → "FIND / FIND"
    - 第一個有，第二個沒有 → "FIND / 空白"
    - 兩個 ID 都沒有 FIND → None（留空）
    """
    # 拆分 TAICCA_ID
    taicca_ids = split_taicca_ids(row['NEW_TAICCA_ID'])
    
    if not taicca_ids:
        return None
    
    find_results = []
    
    # 對每個 ID 查詢
    for taicca_id in taicca_ids:
        # 在 source_df 中查詢該 ID
        matched_rows = source_df[source_df['NEW_TAICCA_ID'] == taicca_id]
        
        if len(matched_rows) > 0:
            # 取第一筆匹配的資料
            matched_row = matched_rows.iloc[0]
            find_value = matched_row.get('未納入書目FIND')
            
            # 檢查是否有值
            if pd.notna(find_value) and str(find_value).strip() != '':
                find_results.append("FIND")
            else:
                find_results.append("空白")
        else:
            # 查不到該 ID
            find_results.append("空白")
    
    # 如果所有結果都是「空白」，返回 None（留空）
    if all(result == "空白" for result in find_results):
        return None
    
    # 用「/」連接結果
    return " / ".join(find_results)

# 測試函數
print("🧪 測試 FIND 欄位查詢:")

# 測試多個 ID 的情況
if len(merge_df) > 0:
    # 找一筆多個 ID 的資料測試
    multi_id_mask = merge_df['NEW_TAICCA_ID'].astype(str).str.contains('/', na=False)
    multi_id_rows = merge_df[multi_id_mask]
    
    if len(multi_id_rows) > 0:
        test_row = multi_id_rows.iloc[0]
        print(f"\n範例 1: 多個 ID")
        print(f"  NEW_TAICCA_ID: {test_row['NEW_TAICCA_ID']}")
        
        # 拆分並查詢每個 ID
        ids = split_taicca_ids(test_row['NEW_TAICCA_ID'])
        print(f"  拆分後: {ids}")
        
        print(f"\n  各 ID 在 df 中的 FIND 欄位:")
        for taicca_id in ids:
            matched = df[df['NEW_TAICCA_ID'] == taicca_id]
            if len(matched) > 0:
                find_val = matched.iloc[0].get('未納入書目FIND')
                has_find = pd.notna(find_val) and str(find_val).strip() != ''
                print(f"    - {taicca_id}: {'有 FIND' if has_find else '無 FIND'} (值: {find_val})")
        
        result = find_find_values_for_row(test_row, df)
        print(f"\n  生成的結果: {result}")
    
    # 測試單一 ID 的情況
    single_id_mask = ~merge_df['NEW_TAICCA_ID'].astype(str).str.contains('/', na=False)
    single_id_rows = merge_df[single_id_mask & merge_df['NEW_TAICCA_ID'].notna()]
    
    if len(single_id_rows) > 0:
        test_row = single_id_rows.iloc[0]
        print(f"\n範例 2: 單一 ID")
        print(f"  NEW_TAICCA_ID: {test_row['NEW_TAICCA_ID']}")
        
        matched = df[df['NEW_TAICCA_ID'] == test_row['NEW_TAICCA_ID']]
        if len(matched) > 0:
            find_val = matched.iloc[0].get('未納入書目FIND')
            has_find = pd.notna(find_val) and str(find_val).strip() != ''
            print(f"  在 df 中的 FIND: {'有' if has_find else '無'} (值: {find_val})")
        
        result = find_find_values_for_row(test_row, df)
        print(f"  生成的結果: {result}")


🧪 測試 FIND 欄位查詢:

範例 1: 多個 ID
  NEW_TAICCA_ID: E253-00004 / E253-00003
  拆分後: ['E253-00004', 'E253-00003']

  各 ID 在 df 中的 FIND 欄位:
    - E253-00004: 無 FIND (值: nan)
    - E253-00003: 無 FIND (值: nan)

  生成的結果: None

範例 2: 單一 ID
  NEW_TAICCA_ID: E253-00010
  在 df 中的 FIND: 無 (值: nan)
  生成的結果: None


In [51]:
print("🔄 開始為 merge_df 填入 FIND 欄位...")
print(f"總共需要處理 {len(merge_df)} 筆資料\n")

# 使用 tqdm 顯示進度條
find_values = []

for idx, row in tqdm(merge_df.iterrows(), total=len(merge_df), desc="處理 FIND"):
    find_result = find_find_values_for_row(row, df)
    find_values.append(find_result)

# 將結果填入 merge_df
merge_df['未納入書目FIND'] = find_values

# 統計結果
total = len(merge_df)
filled = merge_df['未納入書目FIND'].notna().sum()
not_filled = total - filled

print(f"\n✅ FIND 欄位填入完成！")
print(f"  - 總筆數: {total}")
print(f"  - 成功填入: {filled} ({filled/total*100:.1f}%)")
print(f"  - 留空（查無 FIND）: {not_filled} ({not_filled/total*100:.1f}%)")

# 統計不同類型的結果
single_find = (merge_df['未納入書目FIND'] == 'FIND').sum()
multiple_find = merge_df['未納入書目FIND'].astype(str).str.contains('/', na=False).sum()
with_blank = merge_df['未納入書目FIND'].astype(str).str.contains('空白', na=False).sum()

print(f"\n📊 FIND 類型統計:")
print(f"  - 單一 'FIND': {single_find} 筆")
print(f"  - 包含多個結果（有 /）: {multiple_find} 筆")
print(f"  - 包含「空白」: {with_blank} 筆")

# 顯示一些範例
print("\n📋 查看前 10 筆結果:")
print(merge_df[['NEW_TAICCA_ID', '未納入書目FIND']].head(10))


🔄 開始為 merge_df 填入 FIND 欄位...
總共需要處理 9276 筆資料



處理 FIND: 100%|██████████| 9276/9276 [00:03<00:00, 2579.25it/s]


✅ FIND 欄位填入完成！
  - 總筆數: 9276
  - 成功填入: 3500 (37.7%)
  - 留空（查無 FIND）: 5776 (62.3%)

📊 FIND 類型統計:
  - 單一 'FIND': 2783 筆
  - 包含多個結果（有 /）: 717 筆
  - 包含「空白」: 515 筆

📋 查看前 10 筆結果:
             NEW_TAICCA_ID 未納入書目FIND
0  E253-00004 / E253-00003      None
1               E253-00010      None
2               E253-00011      None
3               E253-00050      None
4               E253-05218      None
5               E253-05219      None
6               E253-00493      None
7               E253-00494      None
8               E253-00495      None
9               E253-00496      None


In [52]:
# 修正版本的最終統計（處理 price 欄位的型別問題）

print("=" * 80)
print("📊 最終完整統計（修正版）")
print("=" * 80)

print("\n📅 日期欄位:")
print(f"  - 有 min_publish_date: {merge_df['min_publish_date'].notna().sum()} 筆")
print(f"  - 有 max_publish_date: {merge_df['max_publish_date'].notna().sum()} 筆")

print("\n💰 價格欄位:")
print(f"  - 有 price 資料: {merge_df['price'].notna().sum()} 筆")

# 轉換成數值型，忽略無法轉換的值
price_numeric = pd.to_numeric(merge_df['price'], errors='coerce')
valid_prices = price_numeric.dropna()

if len(valid_prices) > 0:
    print(f"  - 有效數值價格: {len(valid_prices)} 筆")
    print(f"  - 價格範圍: {valid_prices.min():.2f} ~ {valid_prices.max():.2f}")
    print(f"  - 平均價格: {valid_prices.mean():.2f}")
    print(f"  - 中位數價格: {valid_prices.median():.2f}")

print("\n🔍 FIND 欄位:")
print(f"  - 有填入 FIND 相關: {merge_df['未納入書目FIND'].notna().sum()} 筆")
print(f"  - 單一 'FIND': {(merge_df['未納入書目FIND'] == 'FIND').sum()} 筆")

# 統計包含「/」的資料
has_slash = merge_df['未納入書目FIND'].astype(str).str.contains('/', na=False).sum()
has_blank = merge_df['未納入書目FIND'].astype(str).str.contains('空白', na=False).sum()

print(f"  - 包含多個結果（有 /）: {has_slash} 筆")
print(f"  - 包含「空白」: {has_blank} 筆")
print(f"  - 留空（無 FIND）: {merge_df['未納入書目FIND'].isna().sum()} 筆")

# 顯示一些 FIND 類型的範例
print("\n📋 FIND 欄位的各種類型範例:")

# 單一 FIND
single_find = merge_df[merge_df['未納入書目FIND'] == 'FIND']
if len(single_find) > 0:
    print(f"\n  單一 'FIND' 範例 (共 {len(single_find)} 筆):")
    for i, (idx, row) in enumerate(single_find.head(3).iterrows(), 1):
        print(f"    {i}. {row['NEW_TAICCA_ID']}: {row['未納入書目FIND']}")

# FIND / FIND
double_find = merge_df[
    (merge_df['未納入書目FIND'].astype(str).str.contains('/', na=False)) &
    (~merge_df['未納入書目FIND'].astype(str).str.contains('空白', na=False))
]
if len(double_find) > 0:
    print(f"\n  多個 FIND 範例 (共 {len(double_find)} 筆):")
    for i, (idx, row) in enumerate(double_find.head(3).iterrows(), 1):
        print(f"    {i}. {row['NEW_TAICCA_ID']}: {row['未納入書目FIND']}")

# FIND / 空白
mixed_find = merge_df[merge_df['未納入書目FIND'].astype(str).str.contains('空白', na=False)]
if len(mixed_find) > 0:
    print(f"\n  混合型（FIND / 空白）範例 (共 {len(mixed_find)} 筆):")
    for i, (idx, row) in enumerate(mixed_find.head(3).iterrows(), 1):
        print(f"    {i}. {row['NEW_TAICCA_ID']}: {row['未納入書目FIND']}")

print("\n" + "=" * 80)
print("✅ 所有欄位已成功處理！")
print(f"✅ 最終結果已儲存至: final_complete_merged.csv")
print("=" * 80)


📊 最終完整統計（修正版）

📅 日期欄位:
  - 有 min_publish_date: 9275 筆
  - 有 max_publish_date: 9275 筆

💰 價格欄位:
  - 有 price 資料: 9194 筆
  - 有效數值價格: 9194 筆
  - 價格範圍: 0.00 ~ 5460.00
  - 平均價格: 240.14
  - 中位數價格: 210.00

🔍 FIND 欄位:
  - 有填入 FIND 相關: 3500 筆
  - 單一 'FIND': 2783 筆
  - 包含多個結果（有 /）: 717 筆
  - 包含「空白」: 515 筆
  - 留空（無 FIND）: 5776 筆

📋 FIND 欄位的各種類型範例:

  單一 'FIND' 範例 (共 2783 筆):
    1. E253-11459: FIND
    2. E253-11461: FIND
    3. E253-11462: FIND

  多個 FIND 範例 (共 202 筆):
    1. E253-11465 / E253-11458: FIND / FIND
    2. E253-11467 / E253-11460: FIND / FIND
    3. E253-12450 / E253-12443: FIND / FIND

  混合型（FIND / 空白）範例 (共 515 筆):
    1. E253-20387 / E253-11466: 空白 / FIND
    2. E253-20235 / E253-11468: 空白 / FIND
    3. E253-20168 / E253-11469: 空白 / FIND

✅ 所有欄位已成功處理！
✅ 最終結果已儲存至: final_complete_merged.csv


In [53]:
# 修正版：填入原始的 FIND 值（如 FIND_3092）而非只填 "FIND"

def find_find_values_for_row_v2(row, source_df):
    """
    根據 merge_df 的某一列，查詢 source_df (df) 中的 FIND 欄位資料
    
    **修正版**：填入原始的值（如 FIND_3092）
    
    回傳範例：
    - 單一 ID 有 FIND → "FIND_3092"
    - 單一 ID 沒有 FIND → None（留空）
    - 兩個 ID 都有 FIND → "FIND_3092 / FIND_3093"
    - 第一個有，第二個沒有 → "FIND_3092 / 空白"
    - 兩個 ID 都沒有 FIND → None（留空）
    """
    # 拆分 TAICCA_ID
    taicca_ids = split_taicca_ids(row['NEW_TAICCA_ID'])
    
    if not taicca_ids:
        return None
    
    find_results = []
    
    # 對每個 ID 查詢
    for taicca_id in taicca_ids:
        # 在 source_df 中查詢該 ID
        matched_rows = source_df[source_df['NEW_TAICCA_ID'] == taicca_id]
        
        if len(matched_rows) > 0:
            # 取第一筆匹配的資料
            matched_row = matched_rows.iloc[0]
            find_value = matched_row.get('未納入書目FIND')
            
            # 檢查是否有值
            if pd.notna(find_value) and str(find_value).strip() != '':
                # 填入原始的值（如 FIND_3092）
                find_results.append(str(find_value).strip())
            else:
                find_results.append("空白")
        else:
            # 查不到該 ID
            find_results.append("空白")
    
    # 如果所有結果都是「空白」，返回 None（留空）
    if all(result == "空白" for result in find_results):
        return None
    
    # 用「 / 」連接結果
    return " / ".join(find_results)

print("✅ 已定義修正版 find_find_values_for_row_v2 函數")
print("   改動：填入原始的 FIND 值（如 FIND_3092）而非只填 'FIND'")

# 測試修正版函數
print("\n🧪 測試修正版函數:")

# 找一筆有 FIND 的資料來測試
has_find_in_df = df[df['未納入書目FIND'].notna()]
if len(has_find_in_df) > 0:
    test_id = has_find_in_df.iloc[0]['NEW_TAICCA_ID']
    print(f"  測試 ID: {test_id}")
    print(f"  df 中的 FIND 值: {has_find_in_df.iloc[0]['未納入書目FIND']}")
    
    # 創建一個測試 row
    test_row = pd.Series({'NEW_TAICCA_ID': test_id})
    result = find_find_values_for_row_v2(test_row, df)
    print(f"  修正版函數返回: {result}")
    print(f"  ✅ 成功！返回原始值而非 'FIND'")


✅ 已定義修正版 find_find_values_for_row_v2 函數
   改動：填入原始的 FIND 值（如 FIND_3092）而非只填 'FIND'

🧪 測試修正版函數:
  測試 ID: E253-10000
  df 中的 FIND 值: FIND_01
  修正版函數返回: FIND_01
  ✅ 成功！返回原始值而非 'FIND'


In [54]:
print("🔄 重新處理 FIND 欄位（使用修正版函數）...")
print(f"總共需要處理 {len(merge_df)} 筆資料\n")

# 使用修正版函數重新處理
find_values_v2 = []

for idx, row in tqdm(merge_df.iterrows(), total=len(merge_df), desc="處理 FIND (修正版)"):
    find_result = find_find_values_for_row_v2(row, df)
    find_values_v2.append(find_result)

# 將結果填入 merge_df
merge_df['未納入書目FIND'] = find_values_v2

# 統計結果
total = len(merge_df)
filled = merge_df['未納入書目FIND'].notna().sum()
not_filled = total - filled

print(f"\n✅ FIND 欄位重新填入完成！")
print(f"  - 總筆數: {total}")
print(f"  - 成功填入: {filled} ({filled/total*100:.1f}%)")
print(f"  - 留空（查無 FIND）: {not_filled} ({not_filled/total*100:.1f}%)")

# 顯示一些範例來確認原始值有被填入
print("\n📋 查看有 FIND 值的前 10 筆:")
has_find = merge_df[merge_df['未納入書目FIND'].notna()]
if len(has_find) > 0:
    print(has_find[['NEW_TAICCA_ID', '未納入書目FIND']].head(10))
    
    # 確認不是只填 "FIND" 而是原始值
    first_find_value = has_find.iloc[0]['未納入書目FIND']
    print(f"\n✅ 確認：第一筆 FIND 值為 '{first_find_value}'")
    print(f"   這是原始值，不是只填 'FIND' ✅")


🔄 重新處理 FIND 欄位（使用修正版函數）...
總共需要處理 9276 筆資料



處理 FIND (修正版): 100%|██████████| 9276/9276 [00:03<00:00, 2526.53it/s]


✅ FIND 欄位重新填入完成！
  - 總筆數: 9276
  - 成功填入: 3500 (37.7%)
  - 留空（查無 FIND）: 5776 (62.3%)

📋 查看有 FIND 值的前 10 筆:
              NEW_TAICCA_ID              未納入書目FIND
27  E253-11465 / E253-11458  FIND_2320 / FIND_2314
28               E253-11459              FIND_2315
29  E253-11467 / E253-11460  FIND_2322 / FIND_2316
30               E253-11461              FIND_2317
31               E253-11462              FIND_2318
32               E253-11463              FIND_2319
33  E253-20387 / E253-11466         空白 / FIND_2321
34  E253-20235 / E253-11468         空白 / FIND_2323
35  E253-20168 / E253-11469         空白 / FIND_2324
36  E253-20170 / E253-11470         空白 / FIND_2325

✅ 確認：第一筆 FIND 值為 'FIND_2320 / FIND_2314'
   這是原始值，不是只填 'FIND' ✅


In [55]:
# 重新儲存最終結果並顯示詳細統計

print("=" * 80)
print("📊 FIND 欄位修正後的詳細範例")
print("=" * 80)

# 範例 1: 單一 FIND 原始值
single_find_rows = merge_df[
    merge_df['未納入書目FIND'].notna() & 
    (~merge_df['未納入書目FIND'].astype(str).str.contains('/', na=False))
]
if len(single_find_rows) > 0:
    print("\n📋 範例 1: 單一 ID 有 FIND（顯示原始值）")
    print("-" * 80)
    
    for i, (idx, example) in enumerate(single_find_rows.head(3).iterrows(), 1):
        print(f"\n{i}. NEW_TAICCA_ID: {example['NEW_TAICCA_ID']}")
        
        # 查詢 df 中的原始值
        matched = df[df['NEW_TAICCA_ID'] == example['NEW_TAICCA_ID']]
        if len(matched) > 0:
            find_val = matched.iloc[0].get('未納入書目FIND')
            print(f"   df 中的原始 FIND 值: {find_val}")
        
        print(f"   merge_df 中填入的值: {example['未納入書目FIND']}")
        print(f"   ✅ 原始值已正確填入")

# 範例 2: 多個 FIND 原始值
multi_find_rows = merge_df[
    (merge_df['未納入書目FIND'].astype(str).str.contains('/', na=False)) & 
    (~merge_df['未納入書目FIND'].astype(str).str.contains('空白', na=False))
]
if len(multi_find_rows) > 0:
    print("\n📋 範例 2: 多個 ID 都有 FIND（顯示原始值）")
    print("-" * 80)
    
    example = multi_find_rows.iloc[0]
    print(f"\nNEW_TAICCA_ID: {example['NEW_TAICCA_ID']}")
    
    # 拆分並查詢每個 ID 的原始值
    ids = split_taicca_ids(example['NEW_TAICCA_ID'])
    print(f"拆分後: {ids}")
    
    print("\n各 ID 在 df 中的原始 FIND 值:")
    for taicca_id in ids:
        matched = df[df['NEW_TAICCA_ID'] == taicca_id]
        if len(matched) > 0:
            find_val = matched.iloc[0].get('未納入書目FIND')
            print(f"  - {taicca_id}: {find_val if pd.notna(find_val) else '(無)'}")
    
    print(f"\nmerge_df 中填入的值: {example['未納入書目FIND']}")
    print(f"✅ 原始值已正確填入（用 / 連接）")

# 範例 3: 混合型（有原始值 / 空白）
mixed_rows = merge_df[merge_df['未納入書目FIND'].astype(str).str.contains('空白', na=False)]
if len(mixed_rows) > 0:
    print("\n📋 範例 3: 混合型（原始值 / 空白）")
    print("-" * 80)
    
    example = mixed_rows.iloc[0]
    print(f"\nNEW_TAICCA_ID: {example['NEW_TAICCA_ID']}")
    
    ids = split_taicca_ids(example['NEW_TAICCA_ID'])
    print(f"拆分後: {ids}")
    
    print("\n各 ID 在 df 中的 FIND:")
    for taicca_id in ids:
        matched = df[df['NEW_TAICCA_ID'] == taicca_id]
        if len(matched) > 0:
            find_val = matched.iloc[0].get('未納入書目FIND')
            has_find = pd.notna(find_val) and str(find_val).strip() != ''
            print(f"  - {taicca_id}: {find_val if has_find else '(無) → 空白'}")
    
    print(f"\nmerge_df 中填入的值: {example['未納入書目FIND']}")

print("\n" + "=" * 80)

# 重新儲存最終結果
output_file = "final_complete_merged.csv"
merge_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"\n✅ 已重新儲存最終結果至: {output_file}")
print(f"   總筆數: {len(merge_df)}")
print(f"   欄位數: {len(merge_df.columns)}")

# 最終統計
print("\n" + "=" * 80)
print("📊 最終完整統計（修正版）")
print("=" * 80)

print("\n📅 日期欄位:")
print(f"  - 有 min_publish_date: {merge_df['min_publish_date'].notna().sum()} 筆")
print(f"  - 有 max_publish_date: {merge_df['max_publish_date'].notna().sum()} 筆")

print("\n💰 價格欄位:")
price_numeric = pd.to_numeric(merge_df['price'], errors='coerce')
valid_prices = price_numeric.dropna()
print(f"  - 有 price 資料: {merge_df['price'].notna().sum()} 筆")
if len(valid_prices) > 0:
    print(f"  - 有效數值價格: {len(valid_prices)} 筆")
    print(f"  - 價格範圍: {valid_prices.min():.2f} ~ {valid_prices.max():.2f}")
    print(f"  - 平均價格: {valid_prices.mean():.2f}")

print("\n🔍 FIND 欄位:")
print(f"  - 有填入 FIND: {merge_df['未納入書目FIND'].notna().sum()} 筆")
print(f"  - 留空（無 FIND）: {merge_df['未納入書目FIND'].isna().sum()} 筆")

# 統計不同類型的 FIND 值
has_slash = merge_df['未納入書目FIND'].astype(str).str.contains('/', na=False).sum()
has_blank = merge_df['未納入書目FIND'].astype(str).str.contains('空白', na=False).sum()
only_find_values = filled - has_blank  # 純 FIND 值（不含空白）

print(f"  - 純 FIND 值（單一或多個）: {only_find_values} 筆")
print(f"  - 包含多個值（有 /）: {has_slash} 筆")
print(f"  - 包含「空白」: {has_blank} 筆")

print("\n" + "=" * 80)
print("🎉 所有欄位處理完成！FIND 欄位已使用原始值！")
print("=" * 80)


📊 FIND 欄位修正後的詳細範例

📋 範例 1: 單一 ID 有 FIND（顯示原始值）
--------------------------------------------------------------------------------

1. NEW_TAICCA_ID: E253-11459
   df 中的原始 FIND 值: FIND_2315
   merge_df 中填入的值: FIND_2315
   ✅ 原始值已正確填入

2. NEW_TAICCA_ID: E253-11461
   df 中的原始 FIND 值: FIND_2317
   merge_df 中填入的值: FIND_2317
   ✅ 原始值已正確填入

3. NEW_TAICCA_ID: E253-11462
   df 中的原始 FIND 值: FIND_2318
   merge_df 中填入的值: FIND_2318
   ✅ 原始值已正確填入

📋 範例 2: 多個 ID 都有 FIND（顯示原始值）
--------------------------------------------------------------------------------

NEW_TAICCA_ID: E253-11465 / E253-11458
拆分後: ['E253-11465', 'E253-11458']

各 ID 在 df 中的原始 FIND 值:
  - E253-11465: FIND_2320
  - E253-11458: FIND_2314

merge_df 中填入的值: FIND_2320 / FIND_2314
✅ 原始值已正確填入（用 / 連接）

📋 範例 3: 混合型（原始值 / 空白）
--------------------------------------------------------------------------------

NEW_TAICCA_ID: E253-20387 / E253-11466
拆分後: ['E253-20387', 'E253-11466']

各 ID 在 df 中的 FIND:
  - E253-20387: (無) → 空白
  - E253-11466: FIND_232

In [56]:
print("=" * 80)
print("📊 合併資料統計分析")
print("=" * 80)

# 判斷是否為合併資料（NEW_TAICCA_ID 包含「/」）
is_merged = merge_df['NEW_TAICCA_ID'].astype(str).str.contains('/', na=False)

# 統計
total_records = len(merge_df)
merged_records = is_merged.sum()
non_merged_records = total_records - merged_records

print(f"\n📈 總體統計:")
print(f"  - 總筆數: {total_records}")
print(f"  - 合併的資料: {merged_records} 筆 ({merged_records/total_records*100:.1f}%)")
print(f"  - 未合併的資料: {non_merged_records} 筆 ({non_merged_records/total_records*100:.1f}%)")

# 分析合併的資料包含幾個 ID
print(f"\n📊 合併資料的詳細分析:")

merged_df = merge_df[is_merged].copy()

if len(merged_df) > 0:
    # 計算每筆資料包含多少個 ID
    merged_df['id_count'] = merged_df['NEW_TAICCA_ID'].apply(lambda x: len(split_taicca_ids(x)))
    
    # 統計不同 ID 數量的分布
    id_count_stats = merged_df['id_count'].value_counts().sort_index()
    
    print("\n  依據包含的 ID 數量分類:")
    for id_count, count in id_count_stats.items():
        print(f"    - 包含 {id_count} 個 ID: {count} 筆")
    
    # 最多包含幾個 ID
    max_ids = merged_df['id_count'].max()
    max_id_example = merged_df[merged_df['id_count'] == max_ids].iloc[0]
    
    print(f"\n  最多包含的 ID 數: {max_ids} 個")
    print(f"  範例: {max_id_example['NEW_TAICCA_ID']}")
    
    # 計算總共合併了多少筆原始資料
    total_original_ids = merged_df['id_count'].sum()
    print(f"\n💡 這 {merged_records} 筆合併資料，來自原始的 {total_original_ids} 筆資料")
    print(f"   減少了 {total_original_ids - merged_records} 筆重複資料")

# 顯示一些合併資料的範例
print(f"\n📋 合併資料範例（前 10 筆）:")
print("-" * 80)

merged_examples = merge_df[is_merged][['NEW_TAICCA_ID', 'title']].head(10)
for idx, row in merged_examples.iterrows():
    ids = split_taicca_ids(row['NEW_TAICCA_ID'])
    title = row['title'][:50] + '...' if len(str(row['title'])) > 50 else row['title']
    print(f"\n{idx}. {row['NEW_TAICCA_ID']}")
    print(f"   包含 {len(ids)} 個 ID: {ids}")
    print(f"   書名: {title}")

print("\n" + "=" * 80)

# 顯示未合併資料的範例
print(f"\n📋 未合併資料範例（前 5 筆）:")
print("-" * 80)

non_merged_examples = merge_df[~is_merged][['NEW_TAICCA_ID', 'title']].head(5)
for idx, row in non_merged_examples.iterrows():
    title = row['title'][:50] + '...' if len(str(row['title'])) > 50 else row['title']
    print(f"\n{idx}. {row['NEW_TAICCA_ID']}")
    print(f"   書名: {title}")

print("\n" + "=" * 80)


📊 合併資料統計分析

📈 總體統計:
  - 總筆數: 9276
  - 合併的資料: 1993 筆 (21.5%)
  - 未合併的資料: 7283 筆 (78.5%)

📊 合併資料的詳細分析:

  依據包含的 ID 數量分類:
    - 包含 2 個 ID: 1993 筆

  最多包含的 ID 數: 2 個
  範例: E253-00004 / E253-00003

💡 這 1993 筆合併資料，來自原始的 3986 筆資料
   減少了 1993 筆重複資料

📋 合併資料範例（前 10 筆）:
--------------------------------------------------------------------------------

0. E253-00004 / E253-00003
   包含 2 個 ID: ['E253-00004', 'E253-00003']
   書名: (限)Promise：伊藤舞雪寫真集

17. E253-04757 / E253-04756
   包含 2 個 ID: ['E253-04757', 'E253-04756']
   書名: 異鄉的三少爺—林獻堂(「日治時代臺灣人物」臺語繪本系列第三彈！)(電子書)

19. E253-04760 / E253-04759
   包含 2 個 ID: ['E253-04760', 'E253-04759']
   書名: 異數(電子書)

27. E253-11465 / E253-11458
   包含 2 個 ID: ['E253-11465', 'E253-11458']
   書名: 盛世王朝(13)

29. E253-11467 / E253-11460
   包含 2 個 ID: ['E253-11467', 'E253-11460']
   書名: 盛世王朝(15)

33. E253-20387 / E253-11466
   包含 2 個 ID: ['E253-20387', 'E253-11466']
   書名: 盛世王朝(14)

34. E253-20235 / E253-11468
   包含 2 個 ID: ['E253-20235', 'E253-11468']
   書名: 盛世王朝(16)

35. E